In [ ]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import torch, os
from torch.utils.data import Dataset
from ipywidgets import interact 
import seaborn as sns
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import tensorflow as tf
import numpy as np

In [ ]:
torch.__version__

In [ ]:
pd.read_csv('Amazon.csv')

# Loading Data

In [ ]:
train_df, test_df = pd.read_csv('Amazon_Reviews_Train.csv'), pd.read_csv('Amazon_Reviews_Test.csv')

In [ ]:
train_df

In [ ]:
test_df.shape

In [ ]:
from sklearn.model_selection import train_test_split
initial_labeled_set, pool = train_test_split(train_df, test_size=0.95, stratify=train_df['reviews.rating'])

In [ ]:
num_labels = 5
id_to_label = {
    0: "Bad",
    1: "Good",
    2: "Neutral",
    3: "Very Good",
    4: "Excellent"
}

label_to_id = {value : key for key , value in  id_to_label.items()}




# Spliting Data into train_encodings, test_encodings

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)

In [ ]:
train_encodings = tokenizer(train_df['reviews.text'].to_list(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['reviews.text'].to_list(), truncation=True, padding=True)
train_labels = train_df['reviews.rating'].to_list()
test_labels = test_df['reviews.rating'].to_list()

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5, id2label = id_to_label, label2id = label_to_id)

### TrainingArguments, Dataloader, Metrics are needed to construct this Fine-tune the model

In [ ]:
training_args = TrainingArguments(
    output_dir='./BERTModel2',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,  # Reduced number of epochs.
    per_device_train_batch_size=5,  # Reduced batch size for training.
    per_device_eval_batch_size=20,  # Reduced batch size for evaluation.
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    fp16=True,  # Enable mixed precision training.
)


In [ ]:
class DataLoader(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        """
          This construct a dict that is (index position) to encoding pairs.
          Where the Encoding becomes tensor(Encoding), which is an requirements
          for training the model
        """
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        """
        Returns the number of data items in the dataset.

        """
        return len(self.labels)

In [ ]:
def compute_metrics(pred):
    
   
    # Extract true labels from the input object
    labels = pred.label_ids

    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)

    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro',zero_division=1)

    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)

    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

# Calling our Dataloader

In [ ]:
train_dataloader = DataLoader(train_encodings,train_labels)
test_dataloader = DataLoader(test_encodings,test_labels)

In [ ]:
trainer = Trainer(
    #the pre-trained bert model that will be fine-tuned
    model=model,
    #training arguments that we defined above
    args=training_args,
    train_dataset= train_dataloader,
    eval_dataset = test_dataloader,
    compute_metrics= compute_metrics
)

#  Evaluate the prediction accuracy of our model on the dataset without training or fine-tuning it

In [ ]:
predictions = trainer.predict(test_dataloader)


In [ ]:
predicted_labels = predictions.predictions.argmax(axis=1)


In [ ]:
accuracy = accuracy_score(test_df['reviews.rating'].to_list(),predicted_labels)
print('accuracy {}% '.format(np.round(accuracy * 100)))

## An Active Learning Approach Based On Uncertain Sampling

In [ ]:
initial_labeled_set.head() # so we fine_tune with the inital_labeled data

In [ ]:
initial_labeled_set.shape

# 

In [ ]:
def entropy_for_each_row(class_probabilities):
    """ Calculate entropy for each row in the array """
    return -tf.reduce_sum(class_probabilities * tf.math.log(class_probabilities),axis=1)

In [ ]:
def predict_and_calculate_entropy(data):
    
    ''' 
    Predict and Calculate Entropy
    
    This function makes predictions using a pre-trained BERT model, calculates the entropy (uncertainty) of these predictions, 
    and creates a DataFrame containing relevant information.
    
    Args:
        data (DataFrame): A Pandas DataFrame containing text data and associated labels. The DataFrame should have 
        columns 'example' for text data and 'label' for labels.

    Returns:
        final_df (DataFrame): A Pandas DataFrame containing the following columns:
            - 'example': The original text data.
            - 'predicted_Label': The predicted class labels based on the model's predictions.
            - 'predicted_Probability': The maximum predicted probability for each instance.
            - 'Entropy': The calculated entropy (uncertainty) for each instance.
            - 'label': The original labels from the input data.
    ''' 

    
            
    # Sample the Data 
    data_encodings = tokenizer(data['reviews.text'].to_list(), truncation=True, padding=True)
    dataloader = DataLoader(data_encodings, data['reviews.rating'].to_list())

    # Make predictions with class_probabilities and calculate entropy (uncertainty) 
    predictions = trainer.predict(dataloader)
    prediction_probabilities = tf.constant(predictions.predictions)

    # Predicted Labels 
    predicted_labels = predictions.predictions.argmax(axis=1)

    
    # Prediction probabilities, returning the highest probability for each instance
    prediction_probabilities_max = np.amax(prediction_probabilities, axis=1)

    # Calculate entropy for each instance
    entropies = entropy_for_each_row(tf.nn.softmax(prediction_probabilities))

    entropy_df = pd.DataFrame(
        {'reviews.text' : data['reviews.text'].to_list(),
         'predicted_Label': predicted_labels,
         'predicted_Probability': prediction_probabilities_max,
         'Entropy': entropies},
        index=data.index
    )

    final_df = pd.concat([data['reviews.rating'], entropy_df], axis=1)

    return final_df.sort_values(by=['Entropy'],ascending=False)


# Simulated Active Learning

In [ ]:
initial_labeled_set2 = test_df.head(150)

In [ ]:
initial_labeled_set2

In [ ]:
hundreds_rows_pred = predict_and_calculate_entropy(initial_labeled_set2)

In [ ]:
hundreds_rows_pred

# Active Learning

In [ ]:
# Initialize empty lists to store metrics for each iteration
accuracy_list = []
precision_list = []
recall_list = []
loss_list = []
x_labels = []
n = 0

for iteration in range(15):
    n += 10
    print(n)
    initial_labeled_encoding = tokenizer(hundreds_rows_pred[0:n]['reviews.text'].to_list(), truncation=True, padding=True)
    initial_labeled_labels = hundreds_rows_pred[0:n]['reviews.rating'].to_list()
    initial_labeled_dataloader = DataLoader(initial_labeled_encoding,initial_labeled_labels)

   
    trainer = Trainer(
        #the pre-trained bert model that will be fine-tuned
        model=model,
        #training arguments that we defined above
        args=training_args,
        train_dataset= initial_labeled_dataloader,
        eval_dataset = test_dataloader,
        compute_metrics= compute_metrics
    )

    trainer.train()

    metrics = trainer.evaluate()

    print('eval_Accuracy :',metrics['eval_Accuracy'])

    accuracy_list.append(metrics['eval_Accuracy'])
    precision_list.append(metrics['eval_Precision'])
    recall_list.append(metrics['eval_Recall'])
    loss_list.append(metrics['eval_loss'])
    x_labels.append(n)



In [ ]:
# Plot the metrics
plt.figure(figsize=(16, 6))
plt.subplot(1, 3, 1)
plt.plot(x_labels, accuracy_list, marker='o')
plt.title('Accuracy Over Iterations')
plt.xlabel('Number of Samples')
plt.ylabel('Accuracy')

plt.subplot(1, 3, 2)
plt.plot(x_labels, precision_list, marker='o')
plt.title('Precision Over Iterations')
plt.xlabel('Number of Samples')
plt.ylabel('Precision')

plt.subplot(1, 3, 3)
plt.plot(x_labels, recall_list, marker='o')
plt.title('Recall Over Iterations')
plt.xlabel('Number of Samples')
plt.ylabel('Recall')

plt.tight_layout()
plt.show()

In [ ]:
test_df

In [ ]:
predictions = trainer.predict(test_dataloader)
predicted_labels = predictions.predictions.argmax(axis=1)
accuracy = accuracy_score(test_df['reviews.rating'].to_list(),predicted_labels)
print('accuracy {}% '.format(np.round(accuracy * 100)))


In [ ]:
bert_pred_df = pd.DataFrame({'Bert_Pred' : predicted_labels, 'reviews.rating' : test_df['reviews.rating'].to_list() })

In [ ]:
bert_pred_df.to_csv('Amazon_Bert_Pred.csv',index=False)

In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(test_df['reviews.rating'].to_list(),predicted_labels)

# Create a heatmap of the confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix Amazon BERT Prediction')


# Save the confusion matrix as an image
plt.savefig('confusion_matrix.png')
plt.show()

In [ ]:
import csv


# Combine the lists into a list of tuples
data = list(zip(x_labels, accuracy_list, precision_list, recall_list, loss_list))

# Specify the file name
csv_file_name = "Amazon_Label_Accuracies.csv"

# Write the data to a CSV file
with open(csv_file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    csv_writer.writerow(['Labels', 'accuracy', 'precision', 'recall', 'loss'])
    
    # Write the data
    csv_writer.writerows(data)

print(f"CSV file '{csv_file_name}' has been created.")